In [1]:
!pip install -r requirements.txt

In [3]:
import os

from langchain_community.document_loaders import TextLoader
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

%run YandexLLM.ipynb
%reload_ext dotenv
%dotenv
    
api_key = os.getenv('YAGPT_API_KEY')
folder_id = os.getenv('YA_FOLDER_ID')

### Загрузка текстового документа из директории и создание вопроса по документу

In [14]:
loader = TextLoader("./documents/reg_account.txt")
input_doc = loader.load()
#print(type(input_doc),len(input_doc), type(input_doc[0]), input_doc, sep="\n")
query = "Куда должно прийти письмо с подтверждением регистрации?"

<class 'list'>
1
<class 'langchain_core.documents.base.Document'>
[Document(page_content='Инструкция по регистрации аккаунта на сервисе MyApp:\n\n### Шаг 1: Переход на веб-сайт MyApp\n1. Откройте любой веб-браузер на вашем компьютере или мобильном устройстве.\n2. Введите адрес MyApp в адресной строке браузера и нажмите Enter, чтобы перейти на сайт.\n\n### Шаг 2: Нахождение кнопки "Регистрация"\n1. После загрузки сайта MyApp, просмотрите домашнюю страницу и найдите кнопку "Регистрация" или "Создать аккаунт". Обычно эта кнопка располагается в верхнем правом углу страницы или на главной странице.\n\n### Шаг 3: Заполнение регистрационной формы\n1. Щелкните по кнопке "Регистрация". Вас перенаправит на страницу с регистрационной формой.\n2. Заполните поля формы, предоставив необходимую информацию, такую как:\n   - Имя пользователя: Уникальное имя, которое будет использоваться для входа на сервис.\n   - Пароль: Безопасный пароль, состоящий из букв, цифр и специальных символов.\n   - Адрес эле

### Создание prompt шаблона для запроса

In [28]:
# page_content - is a key in langchain structure where to langchain load a document
document_template = PromptTemplate(
    input_variables=["page_content"], #A list of the names of the variables.
    template="{page_content}" # The document template.
)

# The variable name where to put document in final prompt.
document_variable_name = "context"

# Template for prompt
template_override = """
    Представь что ты сотрудник компании MyApp. 
    Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста.
    Текст:
    -----
    {context}
    -----
    Вопрос:
    {query}
"""

# create prompt
prompt = PromptTemplate(
    input_variables=["context", "query"],
    template=template_override
)

### Создание цепочки

In [29]:
# LLM object 
llm = YandexLLM(api_key=api_key, folder_id=folder_id)

# create chain step 1: llm and prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

# create chain step 2: llm, prompt. And add: document, document variable 
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_template,
    document_variable_name=document_variable_name
)

In [30]:
result = chain.run(input_documents=input_doc, query=query)
print(result)

Письмо с подтверждением регистрации должно прийти на адрес электронной почты, который был указан пользователем при регистрации.
